In [1]:
# !pip install backtesting
# !pip install pandas_datareader
# !pip install yfinance

In [8]:
import pandas as pd
import warnings
import yfinance as yf
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

# 必要な追加インポート（例：移動平均やボリンジャーバンド、RSIの計算に使用）
from backtesting.test import SMA, BollingerBands, RSI

warnings.filterwarnings('ignore')

Loading BokehJS ...

ImportError: cannot import name 'BollingerBands' from 'backtesting.test' (/opt/conda/lib/python3.11/site-packages/backtesting/test/__init__.py)

In [ ]:
# # 引数情報
# symbol = "BTC-JPY"  # 仮想通貨の銘柄を指定
# period = "max"      # データ取得範囲を指定

# # データ取得
# Ticker = yf.Ticker(symbol)
# data   = Ticker.history(period=period)

In [ ]:
# ビットコインの価格情報をdfに格納
df = pd.read_csv('./data/BTC_1m.csv')
df

In [5]:
# Dateをdatetime型に変換
df['Datetime'] = pd.to_datetime(df['Datetime'])
df.set_index('Datetime', inplace=True)

# 複数の列を選択してfloat型に変更
columns_to_convert = ['Open', 'High', 'Low', 'Close']
df[columns_to_convert] = df[columns_to_convert].applymap(float)

# Pandasの表示オプションを変更（オプション）
pd.set_option('display.float_format', '{:.2f}'.format)
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2024-02-20 00:00:00+00:00,7780113.00,7780113.00,7780113.00,7780113.00,0,0.00,0.00
2024-02-20 00:01:00+00:00,7780712.00,7780712.00,7780712.00,7780712.00,247201792,0.00,0.00
2024-02-20 00:02:00+00:00,7778306.00,7778306.00,7777378.00,7777378.00,0,0.00,0.00
2024-02-20 00:03:00+00:00,7780024.50,7780024.50,7780024.50,7780024.50,0,0.00,0.00
2024-02-20 00:04:00+00:00,7781959.50,7781959.50,7781959.50,7781959.50,383778816,0.00,0.00
...,...,...,...,...,...,...,...
2024-02-26 08:20:00+00:00,7721123.00,7721123.00,7721123.00,7721123.00,33816576,0.00,0.00
2024-02-26 08:21:00+00:00,7720225.00,7720225.00,7720225.00,7720225.00,1121452032,0.00,0.00
2024-02-26 08:22:00+00:00,7721165.50,7721165.50,7721165.50,7721165.50,444071936,0.00,0.00


In [6]:
# class SmaCrossBollinger(Strategy):
#     n1 = 10  # 短期SMA
#     n2 = 30  # 長期SMA
#     bollinger_period = 20  # ボリンジャーバンドの期間
#     bollinger_dev = 2  # ボリンジャーバンドの標準偏差

#     def init(self):
#         # 移動平均
#         self.sma1 = self.I(SMA, self.data.Close, self.n1)
#         self.sma2 = self.I(SMA, self.data.Close, self.n2)
        
#         # ボリンジャーバンド
#         self.upper, self.middle, self.lower = self.I(BollingerBands, self.data.Close, self.bollinger_period, self.bollinger_dev)

#     def next(self):
#         # SMAクロスオーバー戦略
#         if crossover(self.sma1, self.sma2):
#             self.buy()
#         elif crossover(self.sma2, self.sma1):
#             self.position.close()
        
#         # ボリンジャーバンドを使った追加の条件
#         if self.data.Close[-1] < self.lower[-1]:
#             self.buy()
#         elif self.data.Close[-1] > self.upper[-1]:
#             if self.position:
#                 self.position.close()

In [7]:
class SmaCrossBollinger(Strategy):
    n1 = 10  # 短期SMA
    n2 = 30  # 長期SMA
    bollinger_period = 20  # ボリンジャーバンドの期間
    bollinger_dev = 2  # ボリンジャーバンドの標準偏差
    rsi_period = 14  # RSIの期間
    rsi_overbought = 70  # RSIの買われ過ぎ閾値
    rsi_oversold = 30  # RSIの売られ過ぎ閾値

    def init(self):
        # 移動平均
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
        
        # ボリンジャーバンド
        self.upper, self.middle, self.lower = self.I(BollingerBands, self.data.Close, self.bollinger_period, self.bollinger_dev)

        # RSI
        self.rsi = self.I(RSI, self.data.Close, self.rsi_period)

    def next(self):
        # SMAクロスオーバー戦略
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.position.close()

        # ボリンジャーバンドを使った追加の条件
        if self.data.Close[-1] < self.lower[-1]:
            self.buy()
        elif self.data.Close[-1] > self.upper[-1]:
            if self.position:
                self.position.close()

        # RSIを使った条件
        if self.rsi[-1] < self.rsi_oversold:
            self.buy()
        elif self.rsi[-1] > self.rsi_overbought:
            if self.position:
                self.position.close()

NameError: name 'Strategy' is not defined

In [ ]:
bt = Backtest(
    df, # チャートデータ
    SmaCross, # 売買戦略
    cash=10000000000, # 最初の所持金
    commission=0, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ
)

output = bt.run() # バックテスト実行
#print(output) # 実行結果(データ)
bt.plot() # 実行結果（グラフ）

In [ ]:
output